In [ ]:
#q1

#variables (x,y)
#domains 0<=x<=4  0<=y<=4

#constraints:
#1 start at 1,1 end at 4,4
#2 move only diagonally
#3 avoid obstacles

!pip install ortools
from ortools.sat.python import cp_model

grid_size = 5
start = (1, 1)
target = (4, 4)
obstacles = [(2, 3), (3, 2)]
max_steps = 10

model = cp_model.CpModel()

x = []
y = []

for t in range(max_steps):
    x_var = model.new_int_var(0, grid_size - 1, f'x_{t}')
    y_var = model.new_int_var(0, grid_size - 1, f'y_{t}')
    x.append(x_var)
    y.append(y_var)

model.add(x[0] == start[0])
model.add(y[0] == start[1])

reached_target = []
for t in range(max_steps):
    flag = model.new_bool_var(f'reached_{t}')
    model.add(x[t] == target[0]).OnlyEnforceIf(flag)
    model.add(y[t] == target[1]).OnlyEnforceIf(flag)
    reached_target.append(flag)

model.add(sum(reached_target) >= 1)

for t in range(max_steps):
    for ox, oy in obstacles:
        same_y = model.new_bool_var(f'same_y_{t}_{oy}')
        model.add(y[t] == oy).OnlyEnforceIf(same_y)
        model.add(y[t] != oy).OnlyEnforceIf(same_y.Not())
        model.add(x[t] != ox).OnlyEnforceIf(same_y)

for t in range(max_steps - 1):
    dx = model.new_int_var(-1, 1, f'dx_{t}')
    dy = model.new_int_var(-1, 1, f'dy_{t}')
    model.add(dx == x[t + 1] - x[t])
    model.add(dy == y[t + 1] - y[t])
    model.add_allowed_assignments([dx], [[-1], [1]])
    model.add_allowed_assignments([dy], [[-1], [1]])

model.Minimize(sum(reached_target[t] * t for t in range(max_steps)))

solver = cp_model.CpSolver()
status = solver.solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Path found:\n")
    for t in range(max_steps):
        current_x = solver.value(x[t])
        current_y = solver.value(y[t])
        print(f"Step {t + 1}: ({current_x}, {current_y})")
        if (current_x, current_y) == target:
            break
else:
    print("No valid path found.")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
Path found:

Step 1: (1, 1)
Step 2: (2, 2)
Step 3: (3, 3)
Step 4: (4, 4)


In [ ]:
#q2
from ortools.sat.python import cp_model

input_grid = [
    [0, 1, 1, 0, 0],
    [1, 1, 1, 0, 0],
    [1, 1, 0, 0, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0],
]

rows = len(input_grid)
cols = len(input_grid[0])

model = cp_model.CpModel()
boundary_flags = []

for i in range(rows):
    row_flags = []
    for j in range(cols):
        flag = model.new_bool_var(f"flag_{i}_{j}")
        if input_grid[i][j] == 1:
            edge = False
            if i > 0 and input_grid[i - 1][j] == 0:
                edge = True
            if i < rows - 1 and input_grid[i + 1][j] == 0:
                edge = True
            if j > 0 and input_grid[i][j - 1] == 0:
                edge = True
            if j < cols - 1 and input_grid[i][j + 1] == 0:
                edge = True
            if edge:
                model.add(flag == 1)
            else:
                model.add(flag == 0)
        else:
            model.add(flag == 0)
        row_flags.append(flag)
    boundary_flags.append(row_flags)

all_flags = []
for i in range(rows):
    for j in range(cols):
        all_flags.append(boundary_flags[i][j])

total = model.new_int_var(0, rows * cols, "total")
model.add(total == sum(all_flags))

solver = cp_model.CpSolver()
status = solver.solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Perimeter:", solver.value(total))
    print("Map:")
    for i in range(rows):
        line = ""
        for j in range(cols):
            val = solver.value(boundary_flags[i][j])
            if val == 1:
                line += "B "
            else:
                line += ". "
        print(line)
else:
    print("No solution")


Perimeter: 7
Map:
. B B . . 
B . B . . 
B B . . . 
. B . . . 
. . . . . 


In [ ]:
#q3
from ortools.sat.python import cp_model

distance_matrix = [
    [0, 29, 20, 21, 16, 31, 100, 12, 4, 31],
    [29, 0, 15, 29, 28, 40, 72, 21, 29, 41],
    [20, 15, 0, 15, 14, 25, 81, 9, 23, 27],
    [21, 29, 15, 0, 4, 12, 92, 12, 25, 13],
    [16, 28, 14, 4, 0, 16, 94, 9, 20, 16],
    [31, 40, 25, 12, 16, 0, 95, 24, 36, 3],
    [100, 72, 81, 92, 94, 95, 0, 90, 101, 99],
    [12, 21, 9, 12, 9, 24, 90, 0, 15, 25],
    [4, 29, 23, 25, 20, 36, 101, 15, 0, 35],
    [31, 41, 27, 13, 16, 3, 99, 25, 35, 0]
]

num_cities = 10
model = cp_model.CpModel()

city = []
for i in range(num_cities):
    var = model.new_int_var(0, num_cities - 1, f'city_{i}')
    city.append(var)

model.add_all_different(city)
model.add(city[0] == 0)

flat_distances = []
for i in range(num_cities):
    for j in range(num_cities):
        flat_distances.append(distance_matrix[i][j])

tour_costs = []

for i in range(num_cities - 1):
    index = model.new_int_var(0, num_cities * num_cities - 1, f'index_{i}')
    model.add(index == city[i] * num_cities + city[i + 1])
    dist = model.new_int_var(0, 1000, f'distance_{i}')
    model.add_element(index, flat_distances, dist)
    tour_costs.append(dist)

ret_index = model.new_int_var(0, num_cities * num_cities - 1, 'return_index')
model.add(ret_index == city[num_cities - 1] * num_cities + city[0])
ret_dist = model.new_int_var(0, 1000, 'return_distance')
model.add_element(ret_index, flat_distances, ret_dist)
tour_costs.append(ret_dist)

total_distance = model.new_int_var(0, 10000, 'total_distance')
model.add(total_distance == sum(tour_costs))

model.Minimize(total_distance)

solver = cp_model.CpSolver()
status = solver.solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Tour found:")
    for i in range(num_cities):
        print(f"Step {i + 1}: City {solver.value(city[i])}")
    print(f"Return to: City {solver.value(city[0])}")
    print("Total distance:", solver.value(total_distance))
else:
    print("No tour found.")


Tour found:
Step 1: City 0
Step 2: City 4
Step 3: City 3
Step 4: City 9
Step 5: City 5
Step 6: City 6
Step 7: City 1
Step 8: City 2
Step 9: City 7
Step 10: City 8
Return to: City 0
Total distance: 246


In [ ]:
#Q4
from ortools.sat.python import cp_model

def warehouse_robot_csp():
    model = cp_model.CpModel()

    grid = 6
    robots = 5
    packages = 10
    steps = 15
    max_capacity = 10
    max_battery = 10

    weights = [2, 1, 3, 2, 1, 2, 3, 1, 2, 2]
    origins = [(0, 0), (1, 1), (2, 2), (3, 1), (4, 0), (5, 5), (0, 5), (1, 4), (2, 3), (3, 0)]
    dests = [(5, 0), (4, 5), (0, 4), (1, 3), (3, 5), (2, 0), (5, 2), (3, 3), (1, 0), (0, 2)]
    chargers = [(0, 0), (5, 5)]

    x, y, battery = {}, {}, {}
    for r in range(robots):
        for t in range(steps):
            x[r, t] = model.NewIntVar(0, grid - 1, f"x_{r}_{t}")
            y[r, t] = model.NewIntVar(0, grid - 1, f"y_{r}_{t}")
            battery[r, t] = model.NewIntVar(0, max_battery, f"bat_{r}_{t}")
        model.Add(battery[r, 0] == max_battery)

    assigned = {}
    for p in range(packages):
        assigned[p] = model.NewIntVar(0, robots - 1, f"assign_{p}")

    for r in range(robots):
        load = []
        for p in range(packages):
            use = model.NewBoolVar(f"robot_{r}_has_p{p}")
            model.Add(assigned[p] == r).OnlyEnforceIf(use)
            model.Add(assigned[p] != r).OnlyEnforceIf(use.Not())
            weight_use = model.NewIntVar(0, max_capacity, f"weight_r{r}_p{p}")
            model.AddMultiplicationEquality(weight_use, [use, weights[p]])
            load.append(weight_use)
        model.Add(sum(load) <= max_capacity)

    for r in range(robots):
        for t in range(1, steps):
            dx = model.NewIntVar(-1, 1, f"dx_{r}_{t}")
            dy = model.NewIntVar(-1, 1, f"dy_{r}_{t}")
            model.Add(dx == x[r, t] - x[r, t - 1])
            model.Add(dy == y[r, t] - y[r, t - 1])
            model.AddAbsEquality(model.NewIntVar(0, 1, ""), dx)
            model.AddAbsEquality(model.NewIntVar(0, 1, ""), dy)

            is_at_charger = model.NewBoolVar(f"at_charger_{r}_{t}")
            conditions = []
            for cx, cy in chargers:
                cx_match = model.NewBoolVar("")
                cy_match = model.NewBoolVar("")
                model.Add(x[r, t] == cx).OnlyEnforceIf(cx_match)
                model.Add(x[r, t] != cx).OnlyEnforceIf(cx_match.Not())
                model.Add(y[r, t] == cy).OnlyEnforceIf(cy_match)
                model.Add(y[r, t] != cy).OnlyEnforceIf(cy_match.Not())
                match = model.NewBoolVar("")
                model.AddBoolAnd([cx_match, cy_match]).OnlyEnforceIf(match)
                model.AddBoolOr([cx_match.Not(), cy_match.Not()]).OnlyEnforceIf(match.Not())
                conditions.append(match)
            model.AddMaxEquality(is_at_charger, conditions)

            drop = model.NewIntVar(0, 1, f"bat_drop_{r}_{t}")
            model.Add(drop == 0).OnlyEnforceIf(is_at_charger)
            model.Add(drop == 1).OnlyEnforceIf(is_at_charger.Not())
            model.Add(battery[r, t] == battery[r, t - 1] - drop)

    for t in range(steps):
        for r1 in range(robots):
            for r2 in range(r1 + 1, robots):
                sx = model.NewBoolVar("")
                sy = model.NewBoolVar("")
                model.Add(x[r1, t] == x[r2, t]).OnlyEnforceIf(sx)
                model.Add(x[r1, t] != x[r2, t]).OnlyEnforceIf(sx.Not())
                model.Add(y[r1, t] == y[r2, t]).OnlyEnforceIf(sy)
                model.Add(y[r1, t] != y[r2, t]).OnlyEnforceIf(sy.Not())
                model.AddBoolOr([sx.Not(), sy.Not()])

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        for r in range(robots):
            print(f"\nRobot {r} path:")
            for t in range(steps):
                print(f"  Time {t}: ({solver.Value(x[r, t])}, {solver.Value(y[r, t])}), Battery: {solver.Value(battery[r, t])}")
        print("\nAssignments:")
        for p in range(packages):
            print(f"  Package {p} → Robot {solver.Value(assigned[p])}")
    else:
        print("No feasible solution found.")

warehouse_robot_csp()



Robot 0 path:
  Time 0: (4, 4), Battery: 10
  Time 1: (5, 5), Battery: 10
  Time 2: (4, 4), Battery: 9
  Time 3: (3, 3), Battery: 8
  Time 4: (2, 4), Battery: 7
  Time 5: (1, 3), Battery: 6
  Time 6: (2, 2), Battery: 5
  Time 7: (2, 3), Battery: 4
  Time 8: (3, 3), Battery: 3
  Time 9: (4, 4), Battery: 2
  Time 10: (5, 5), Battery: 2
  Time 11: (5, 5), Battery: 2
  Time 12: (5, 5), Battery: 2
  Time 13: (4, 4), Battery: 1
  Time 14: (3, 3), Battery: 0

Robot 1 path:
  Time 0: (1, 0), Battery: 10
  Time 1: (0, 0), Battery: 10
  Time 2: (0, 0), Battery: 10
  Time 3: (0, 0), Battery: 10
  Time 4: (0, 0), Battery: 10
  Time 5: (0, 1), Battery: 9
  Time 6: (0, 2), Battery: 8
  Time 7: (0, 3), Battery: 7
  Time 8: (0, 4), Battery: 6
  Time 9: (0, 4), Battery: 5
  Time 10: (0, 3), Battery: 4
  Time 11: (1, 2), Battery: 3
  Time 12: (0, 1), Battery: 2
  Time 13: (1, 0), Battery: 1
  Time 14: (2, 0), Battery: 0

Robot 2 path:
  Time 0: (0, 1), Battery: 10
  Time 1: (0, 1), Battery: 9
  Time 2:

In [ ]:
#q5
from ortools.sat.python import cp_model

def solve_sudoku(grid):
    model = cp_model.CpModel()

    SIZE = 9
    SUBGRID = 3
    PRIMES = [2, 3, 5, 7]

    cells = []
    for r in range(SIZE):
        row = []
        for c in range(SIZE):
            row.append(model.NewIntVar(1, 9, f'cell_{r}_{c}'))
        cells.append(row)

    for r in range(SIZE):
        for c in range(SIZE):
            if grid[r][c] != 0:
                model.Add(cells[r][c] == grid[r][c])

    for r in range(SIZE):
        model.AddAllDifferent(cells[r])

    for c in range(SIZE):
        col = []
        for r in range(SIZE):
            col.append(cells[r][c])
        model.AddAllDifferent(col)

    for br in range(0, SIZE, SUBGRID):
        for bc in range(0, SIZE, SUBGRID):
            block = []
            for i in range(SUBGRID):
                for j in range(SUBGRID):
                    block.append(cells[br + i][bc + j])
            model.AddAllDifferent(block)

    main_diag = []
    anti_diag = []
    for i in range(SIZE):
        main_diag.append(cells[i][i])
        anti_diag.append(cells[i][SIZE - 1 - i])

    diag1_sum = model.NewIntVar(0, 81, "diag1_sum")
    diag2_sum = model.NewIntVar(0, 81, "diag2_sum")

    model.Add(diag1_sum == sum(main_diag))
    model.Add(diag2_sum == sum(anti_diag))

    model.AddModuloEquality(0, diag1_sum, 3)
    model.AddModuloEquality(0, diag2_sum, 3)

    for r in range(SIZE):
        for c in range(SIZE):
            for dr, dc in [(0, 1), (1, 0)]:
                nr = r + dr
                nc = c + dc
                if nr < SIZE and nc < SIZE:
                    for prime in PRIMES:
                        cond1 = model.NewBoolVar("")
                        cond2 = model.NewBoolVar("")
                        model.Add(cells[r][c] == prime).OnlyEnforceIf(cond1)
                        model.Add(cells[r][c] != prime).OnlyEnforceIf(cond1.Not())
                        model.Add(cells[nr][nc] == prime).OnlyEnforceIf(cond2)
                        model.Add(cells[nr][nc] != prime).OnlyEnforceIf(cond2.Not())
                        both_prime = model.NewBoolVar("")
                        model.AddBoolAnd([cond1, cond2]).OnlyEnforceIf(both_prime)
                        model.AddBoolOr([cond1.Not(), cond2.Not()]).OnlyEnforceIf(both_prime.Not())

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        for r in range(SIZE):
            row = []
            for c in range(SIZE):
                row.append(solver.Value(cells[r][c]))
            print(row)
    else:
        print("No solution found.")

sudoku_grid = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0]
]


solve_sudoku(sudoku_grid)


[7, 3, 9, 4, 5, 1, 8, 6, 2]
[8, 1, 4, 6, 2, 7, 5, 3, 9]
[6, 5, 2, 3, 8, 9, 1, 7, 4]
[2, 7, 3, 1, 9, 5, 4, 8, 6]
[9, 6, 5, 7, 4, 8, 3, 2, 1]
[1, 4, 8, 2, 6, 3, 9, 5, 7]
[3, 9, 1, 8, 7, 6, 2, 4, 5]
[5, 2, 6, 9, 3, 4, 7, 1, 8]
[4, 8, 7, 5, 1, 2, 6, 9, 3]
